In [3]:
import redis
import random
import math

def create_clients(r):
    lat_from, long_from, lat_to, long_to = tehran_latlong()
    itr = 50000
    for i in range(itr):
        lat = from_range(lat_from, lat_to, random.random())
        long = from_range(long_from, long_to, random.random())
        # r.geoadd("shops", long, lat, str(i))
        r.geoadd("shops", [long,lat,str(i)])

def tehran_latlong():
    lat_from = 35.679292
    long_from = 51.306170
    lat_to = 35.764646
    long_to = 51.487273
    return lat_from, long_from, lat_to, long_to

def from_range(from_value, to_value, x):
    return x * (to_value - from_value) + from_value

# Usage example
r = redis.Redis(host='127.0.0.1', port=6379, db=0)
create_clients(r)

In [19]:
import redis
from redis.commands.search.field import GeoField
from redis.commands.search.query import Query

def find_near(r, lat, long, radius):
    res = r.geosearch(
        "shops",
        longitude=long,
        latitude=lat,
        radius=radius,
        unit="m",
        withcoord=True,
        count=50,
        sort="asc"
    )
    return res

# Usage example
r = redis.Redis(host='127.0.0.1', port=6379, db=0)
nearby_locations = find_near(r, 35.679292, 51.306170, 5000)

# print(len(nearby_locations[0]))

for loc in nearby_locations:
    print(f"Name: {loc[0]}, Coordinates: {loc[1]}")

Name: b'47528', Coordinates: (51.30648225545883, 35.679593041967514)
Name: b'30485', Coordinates: (51.30650371313095, 35.67995297237214)
Name: b'10356', Coordinates: (51.307367384433746, 35.6796310627849)
Name: b'13444', Coordinates: (51.30709379911423, 35.68001634040112)
Name: b'19797', Coordinates: (51.30732446908951, 35.67996057653561)
Name: b'32644', Coordinates: (51.30706697702408, 35.68035599303648)
Name: b'28067', Coordinates: (51.307914555072784, 35.67948911839998)
Name: b'24043', Coordinates: (51.30651444196701, 35.68069818039299)
Name: b'36924', Coordinates: (51.30783945322037, 35.67987946545852)
Name: b'35540', Coordinates: (51.30627304315567, 35.68082998589327)
Name: b'37830', Coordinates: (51.30810767412186, 35.67938012539013)
Name: b'20478', Coordinates: (51.3082417845726, 35.67930661847651)
Name: b'43762', Coordinates: (51.30803257226944, 35.68006196538199)
Name: b'1718', Coordinates: (51.307651698589325, 35.680650020690955)
Name: b'2245', Coordinates: (51.30829006433487

In [34]:
from flask import Flask, request

app = Flask(__name__)
if __name__ == '__main__':
    r = redis.Redis(host='127.0.0.1', port=6379, db=0)

    @app.route('/', methods=['GET'])
    def benchmark():
        lat_from, long_from, lat_to, long_to = tehran_latlong()
        lat = from_range(lat_from, lat_to, random.random())
        long = from_range(long_from, long_to, random.random())
        items = find_near(r, lat, long, 250)
        result = ""
        i=1
        for item in items:
            result+=f"\n{i}){item[0]}\t {item[1][0]}, {item[1][1]}"
            i+=1
        # result='\n'.join([f"{item[0]}\t {item[1][0]}, {item[1][1]}" for item in items])
        return result, 200, {'Content-Type': 'text/plain'}

    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [24/Jun/2024 10:15:02] "GET / HTTP/1.1" 200 -
